<span style="color:orange">
 <h2> Databricks-Assignemnt 2 (Basic Data Cleaning Transformation)

In [0]:
from pyspark.sql.functions import *

In [0]:
df = spark.read.csv("dbfs:/FileStore/shared_uploads/aakashpavale8877@gmail.com/EmployeeData_HhTLiz9LdC_2JpHdxGvrY-1.csv",header=True, inferSchema=True, sep =',')
df.printSchema()
df.show()

root
 |-- id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- creationDate: timestamp (nullable = true)
 |-- bonus: double (nullable = true)

+---+----------+---------+--------------------+------+------+-------------------+--------+
| id|first_name|last_name|               email|gender|salary|       creationDate|   bonus|
+---+----------+---------+--------------------+------+------+-------------------+--------+
|  1|    Valene|   Ingley|vingley0@livejour...|Female| 44104|1957-09-09 16:44:40|    null|
|  2|  Lynnelle|    Hurll| lhurll1@answers.com|Female|112411|1907-05-10 17:38:56|    null|
|  3|   Miranda|    Train|   mtrain2@imgur.com|Female| 91073|1941-01-24 16:05:23|12875.54|
|  4|    Dulsea|     Foss|dfoss3@dagondesig...|Female|193291|1942-05-09 20:59:39|    null|
|  5|    Anatol|  Dunklee| adunklee4@goog

<b> Find the count of duplicate employee records in the input file (based on id)?

In [0]:
# Write your code using the Spark Function
#did not find any duplicated as per id so did it on first name and lasst name basis
dup_count = df.groupBy('first_name','last_name').agg(count('id').alias('countofids')).orderBy(desc_nulls_last('countofids'))
result = dup_count.filter(dup_count.countofids > 1).select('first_name','last_name')
result.show()

+----------+---------+
|first_name|last_name|
+----------+---------+
|   Giacomo|     null|
+----------+---------+



<b> Find out how many records have Gender value missing.

In [0]:
# Write your code using the Spark Function
gender_missing = df.where(df.gender.isNull()).select(count('id').alias("Countofidswithgenderasnull"))
gender_missing.show()

+--------------------------+
|Countofidswithgenderasnull|
+--------------------------+
|                        28|
+--------------------------+



d
<b> Are there any missing values in the "bonus" field? If so, filled them defualt bonus 100.

In [0]:
# Write your code using the Spark Function
fill_cols_vals = {"bonus": 100}
df1 = df.na.fill(fill_cols_vals)
df1.show()

+---+----------+---------+--------------------+------+------+-------------------+--------+
| id|first_name|last_name|               email|gender|salary|       creationDate|   bonus|
+---+----------+---------+--------------------+------+------+-------------------+--------+
|  1|    Valene|   Ingley|vingley0@livejour...|Female| 44104|1957-09-09 16:44:40|   100.0|
|  2|  Lynnelle|    Hurll| lhurll1@answers.com|Female|112411|1907-05-10 17:38:56|   100.0|
|  3|   Miranda|    Train|   mtrain2@imgur.com|Female| 91073|1941-01-24 16:05:23|12875.54|
|  4|    Dulsea|     Foss|dfoss3@dagondesig...|Female|193291|1942-05-09 20:59:39|   100.0|
|  5|    Anatol|  Dunklee| adunklee4@google.de|  Male| 22175|1950-07-26 16:28:00| 1432.12|
|  6|     Baily|   Antony| bantony5@sfgate.com|  Male|127337|1913-10-14 11:25:33|   100.0|
|  7|    Eunice|   Cardus|ecardus6@scientif...|Female|136574|1901-11-02 15:09:47|38676.94|
|  8|  Aubrette|  Lippett| alippett7@nifty.com|Female|165713|1919-12-16 08:20:42|60150.66|

d
<b> Are there any employees with negative salary or bonus amounts in the input file? If so, how many?

In [0]:
# Write your code using the Spark Function
negative_values = df.where("salary < 0 or bonus < 0").select('id','first_name','last_name','salary','bonus')
negative_values.show()

+---+----------+---------+------+-----+
| id|first_name|last_name|salary|bonus|
+---+----------+---------+------+-----+
+---+----------+---------+------+-----+



d
<b> Replace all the null/emtpy value in email column with admin@azurelib.com

In [0]:
# Write your code using the Spark Function
df2 = df.na.fill("admin@azurelib.com", subset=["email"])
df2.where("email = 'admin@azurelib.com'").show()

+----+----------+-----------+------------------+------+------+-------------------+--------+
|  id|first_name|  last_name|             email|gender|salary|       creationDate|   bonus|
+----+----------+-----------+------------------+------+------+-------------------+--------+
|  21|    Kristo|    Giraudy|admin@azurelib.com|  Male|152116|1962-08-16 21:27:30| 5982.54|
|  28| Sallyanne|  Iacobucci|admin@azurelib.com|Female|  2339|1939-05-10 11:30:31| 66003.0|
|  29|      null|     Nibley|admin@azurelib.com|Female|165764|2019-08-30 11:23:27|93953.74|
|  46|Kristoforo|     Sowray|admin@azurelib.com|  Male|131676|1959-03-14 16:01:23|27022.26|
|  64|   Cordell|Baszkiewicz|admin@azurelib.com|  Male| 89429|1998-04-26 03:44:59|50640.06|
|  67|    Aloise|   Brombell|admin@azurelib.com|Female| 51529|1987-02-16 16:02:18|    null|
|  70|    Lianne|       Boam|admin@azurelib.com|Female| 13695|1981-04-08 20:31:44|    null|
|  78|   Jillian|    Chasmar|admin@azurelib.com|Female| 18805|2019-01-07 02:06:4

d
<b> Remove all the records where any record has any null values. Find out the total count of the records now.

In [0]:
# Write your code using the Spark Function
current_row_count = df.select(count('*').alias('preCleaningRowCount'))
current_row_count.show()
df_remove_null = df.na.drop("any")
post_row_count = df_remove_null.select(count('id').alias('postCleaningRowCount'))
post_row_count.show()

+-------------------+
|preCleaningRowCount|
+-------------------+
|               1000|
+-------------------+

+--------------------+
|postCleaningRowCount|
+--------------------+
|                 512|
+--------------------+

